In [ ]:
import os
import glob
import json
import pprint

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

from tqdm import tqdm
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction import FeatureHasher

In [ ]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r",encoding='ISO-8859-1') as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"], probability=True)
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1, max_iter=25000)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print(f"{model}정확도", model.score(X_test, y_test))


## 특징 벡터 생성 예시
- PEMINER 정보는 모두 수치형 데이터이므로 특별히 가공을 하지 않고 사용 가능
- EMBER, PESTUDIO 정보는 가공해서 사용해야 할 특징들이 있음 (e.g. imports, exports 등의 문자열 정보를 가지는 데이터)
- 수치형 데이터가 아닌 데이터(범주형 데이터)를 어떻게 가공할 지가 관건 >> 인코딩 (e.g. 원핫인코딩, 레이블인코딩 등)

In [ ]:
from sklearn.feature_extraction import FeatureHasher

class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        '''
            전체 데이터 사용        
        '''
        
        self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        return self.vector
    

class EmberParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()
    
    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]
        return vector

    ##############################
    # 직접 추가한 특징 추출 함수 #   
    ##############################

    #import하는 라이브러리등 feature 해시로 추출 
    def get_imports_info(self):
        data = self.report["imports"]
        vector = []
        libraries = list(set([l.lower() for l in data]))
        vector.extend(FeatureHasher(256, input_type="string").transform([libraries]).toarray()[0]) # libraries_hashed

        imports = [lib.lower() + ':' + e for lib, elist in data.items() for e in elist]
        vector.extend(FeatureHasher(1024, input_type="string").transform([imports]).toarray()[0]) # imports_hashed

        return vector

    #악성코드의 경우 export를 많이 할것으로 예상됨
    def get_exports_info(self):
        exports = self.report["exports"]
        vector = []
        vector.extend(FeatureHasher(128, input_type="string").transform([exports]).toarray()[0]) # exports_hashed
        return vector

    # header의 size가 지나치게 많거나
    # version이 잘 못 되는 등의 상황이 의심됨
    def get_header_info(self): 
        header = self.report['header']
        vector = [
            header['coff']['timestamp'],
            header['optional']['major_image_version'],
            header['optional']['minor_image_version'],
            header['optional']['major_linker_version'],
            header['optional']['minor_linker_version'],
            header['optional']['major_operating_system_version'],
            header['optional']['major_subsystem_version'],
            header['optional']['sizeof_code'],
            header['optional']['sizeof_headers'],
            header['optional']['sizeof_heap_commit'],
        ]
        return vector
    
    # 파일의 정보에서 악성파일의 경우 비정상적인 상황이 있을 확률이 높음.
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]
        return vector

    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        vector += self.get_histogram_info()
        vector += self.get_string_info()
        
        
        '''
            특징 추가
        '''
        vector += self.get_general_file_info()
        vector += self.get_header_info()
        vector += self.get_exports_info()
        vector += self.get_imports_info()
        return vector
    
class PestudioParser:

    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []

    '''
        사용할 특징을 선택하여 벡터화 할 것을 권장
    '''
    
    # libraries의 blacklist = "x"인 비율
    def get_libraries_info(self):
        try:
            libraries = self.report["image"]["libraries"]
        except KeyError:
            return [-1]
        if libraries == "n/a" or len(libraries) == 1:
            return [0]
        try:
            vector = [sum(1 for i in libraries["library"] if i["@blacklist"] == "x") / len(libraries["library"])]
        except TypeError:
            vector = [1 if libraries["library"]["@blacklist"] == "x" else 0]
        return vector

    # imports의 blacklist = "x"인  비율
    def get_imports_info(self):
        try:
            imports = self.report["image"]["imports"]
        except KeyError:
            return [-1]
        if imports == "n/a" or len(imports) == 1:
            return [0]
        try:
            vector = [sum(1 for i in imports["import"] if i["@blacklist"] == "x") / len(imports["import"])]
        except TypeError:
            vector = [1 if imports["import"]["@blacklist"] == "x" else 0]
        return vector

    # exports의 export 요소 개수 추출
    def get_exports_info(self):
        try:
            exports = self.report["image"]["exports"]
        except KeyError:
            return [-1]
        if exports == "n/a":
            return [0]
        vector = [len(exports["export"])]
        return vector

    #인증서가 있으면 0 없다면 1
    def get_certificate(self):
      try:
        cert = self.report['image']['certificate']
        if cert == 'n/a':
          return [0]
        else:
          return [1]
      except:
        return [0]
    

    
    def process_report(self):

        vector = []
        vector += self.get_libraries_info()
        vector += self.get_imports_info()
        vector += self.get_exports_info()
        vector += self.get_certificate()


        return vector

In [ ]:
# 파일 목록 불러오기
file_list = os.listdir("/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/EMBER/학습데이터")
very_list = os.listdir("/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/EMBER/검증데이터")
test_list = os.listdir("/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/EMBER/테스트데이터")

## 학습데이터 구성
- 특징 벡터 구성은 2차원이 되어야함 e.g.  [vector_1, vector_2, ..., vector_n]

- 각 벡터는 1차원 리스트, 벡터 크기는 모두 같아야함

In [ ]:

# 데이터의 특징 벡터 모음(2차원 리스트) : X
# 데이터의 레이블 모음(1차원 리스트) : y
X, y = [], []

papa = "/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/"
label_table = read_label_csv(f"{papa}학습데이터_정답.csv")
X, y = [], []
l = os.listdir('/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/EMBER/학습데이터')
for fname in l:
    feature_vector = []
    label = label_table[fname.split('.')[0]]
    for data in ["PEMINER", "EMBER"]:
        path = f"/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/{data}/학습데이터/{fname}"
 
        if data == "PEMINER":
              feature_vector += PeminerParser(path).process_report()
        else :
              feature_vector += EmberParser(path).process_report()

            # print(path)
    X.append(feature_vector)
    y.append(label)

In [ ]:
X_val, y_val = [], []
l = os.listdir('/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/EMBER/검증데이터')
label_table = read_label_csv(f"{papa}검증데이터_정답.csv")
for fname in l:
    feature_vector = []
    label = label_table[fname.split('.')[0]]
    for data in ["PEMINER", "EMBER"]:
        path = f"/Users/bumseok/workspace/2021_Information_protection/Project 2/데이터/{data}/검증데이터/{fname}"
 
        if data == "PEMINER":
              feature_vector += PeminerParser(path).process_report()
        else :
              feature_vector += EmberParser(path).process_report()

            # print(path)
    X_val.append(feature_vector)
    y_val.append(label)

np.asarray(X_val).shape, np.asarray(y_val).shape

## 학습 및 검증

In [ ]:
# 학습
model_name = ["rf", "lgb", "dt", "svm", "lr", "knn", "adaboost", "mlp"]
models = []
for model in tqdm(model_name):
    clf = train(X, y, model)
    models.append(clf)
    
# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in tqdm(models):
    evaluate(X_val, y_val, model)

In [ ]:
# 학습
models = [] # 여기에 dt lgb svm lr knn adaboost mlp 추가해서 모델 설정
for model in tqdm(["rf", "lgb"]):
    clf = train(X, y, model)
    models.append(clf)
    
# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in tqdm(models):
    evaluate(X_val, y_val, model)

## 앙상블 예제

In [ ]:
def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
	
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
    
    # Soft Voting
    # https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting
    predicts = []
    for i in tqdm(range(len(X))):
        probs = []
        for model in models:
            prob = model.predict_proba(X)[i][1]
            probs.append(prob)
        predict = 1 if np.mean(probs) >= 0.5 else 0
        predicts.append(predict)
        
    print("정확도", accuracy_score(y, predicts))

In [ ]:
ensemble_result(X_val, y_val, models)

## 특징 선택 예제 (RFE 알고리즘 사용)

In [ ]:
def select_feature(X, y, model):
    '''
        주어진 특징 벡터에서 특정 알고리즘 기반 특징 선택
        
        본 예제에서는 RFE 알고리즘 사용
        https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE.fit_transform
        
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param model: 문자열, 특징 선택에 사용할 머신러닝 알고리즘
    '''
    
    model = load_model(model=model, random_state=SEED)
    rfe = RFE(estimator=model)
    return rfe.fit_transform(X, y)



In [45]:
selected_X = select_feature(X, y, "rf")

In [46]:
new_model = train(selected_X, y, "rf")

In [50]:
# evaluate(selected_X, y_val, new_model)
print(y.shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# X_val = np.asarray(X_val)
# y_val = np.asarray(y_val)

mode=[]
mode.append(new_model)
for model in mode:
    evaluate(X_val,y_val,model)